In [45]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objs as go
import pandas as pd
from urllib.request import urlopen

import datetime

import os
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
     map_base = json.load(response)

In [46]:
counties = pd.read_csv('counties.csv',sep = ',')
counties.columns = [name.strip() for name in counties.columns.values]
counties.dropna(subset = ['FIPS #'],inplace= True)
counties['FIPS #'] = (counties['FIPS #']+48000).astype(int).astype(str)
counties['County Name'] = counties['County Name'].str.strip()
cty_lst = counties['County Name'].tolist()
counties = pd.Series(counties['FIPS #'].values,index=counties['County Name']).to_dict()

In [47]:
dataframes = {}

In [48]:
available_indicators = ['PM2.5','Ozone','CO','NO2']
current_indicator = ['Ozone']
current_date = datetime.datetime(2020,1,1)

In [49]:
base = datetime.datetime.today()
base = base.replace(hour=0, minute=0, second=0)
numdays = (base - datetime.datetime.fromisoformat('2020-01-01')).days
base = datetime.datetime(2020,1,1)+datetime.timedelta(days=numdays)
date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]
date_list.insert(0,datetime.datetime(2020,1,1))
z = 0
for param in available_indicators:
    df = pd.read_csv('difference/'+param+'.csv', sep = ',',parse_dates = ['Date'], index_col = 'Date')
    df.drop('Unnamed: 0',axis = 1,inplace = True)
    df['FIPS #'] = df['County'].map(counties)
    df.reset_index(inplace = True)
    df = df.groupby(['County','FIPS #','Date']).mean().reset_index()
    to_append = []
    for date in date_list:
        temp_df = df[df['Date'] == date]
        cty_col = temp_df['County'].tolist()
        for cty in cty_lst:
            if cty not in cty_col:
                to_append.append({'County':cty,'Date':date,'value':0,'FIPS #':counties[cty]})
    before = df.copy()
    df = df.append(pd.DataFrame(to_append))
    #print(df==before)
    z = to_append
    to_append = []
    dataframes[param] = df

In [50]:
pd.DataFrame(z)

,County,Date,FIPS #,value
0,Anderson,2020-01-01,48001,0
1,Andrews,2020-01-01,48003,0
2,Angelina,2020-01-01,48005,0
3,Aransas,2020-01-01,48007,0
4,Archer,2020-01-01,48009,0
5,Armstrong,2020-01-01,48011,0
6,Atascosa,2020-01-01,48013,0
7,Austin,2020-01-01,48015,0
8,Bailey,2020-01-01,48017,0
9,Bandera,2020-01-01,48019,0


In [51]:
s =dataframes['PM2.5']
s[s['Date'] == datetime.datetime(2020,1,1)]

,County,Date,FIPS #,value
0,Bexar,2020-01-01,48029,0.694991
191,Cameron,2020-01-01,48061,-0.873669
382,Dallas,2020-01-01,48113,5.985331
573,Denton,2020-01-01,48121,5.405116
764,Ector,2020-01-01,48135,-2.132037
955,El Paso,2020-01-01,48141,-1.207053
1146,Ellis,2020-01-01,48139,1.242070
1337,Galveston,2020-01-01,48167,1.149679
1528,Harris,2020-01-01,48201,33.736582
1719,Harrison,2020-01-01,48203,1.713416


In [52]:
master_df = dataframes[current_indicator[-1]]
df = master_df[master_df['Date'] == current_date]
# print(df)
# trace= go.Choroplethmapbox(geojson=map_base,
#                        locations=df['FIPS #'].astype(str),
#                        z=df['value'],
#                        colorscale='ice',
#                        colorbar_thickness=20,
#                        text = df['County'],
#                        marker_line_color='white',customdata = df['FIPS #'], hovertemplate = '<b>County</b>: <b>%{text}</b>'+
#                                         '<br> <b>Val </b>: %{z}<br>'+  '<extra></extra>'
#                     )
# fig= go.Figure(data = trace)
# mapboxt = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw'                           

# fig.update_layout(title_text= 'Texas Heatmap',
#                   title_x=0.5, width = 700, height=700,
#                   mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
#                                  accesstoken= mapboxt,style = 'basic',
#                                  zoom=4.5,
#                                ));

In [ ]:
app = dash.Dash()
app.layout = html.Div([
    html.H1('Texas Air Quality Heatmap'),
    html.Div(id='text-content'),
    dcc.Dropdown(
                id='parameter',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Ozone'),
    dcc.Graph(id='map'),dcc.Slider(
        id='year-slider',
        min=0,
        max=numdays,
        value=0,
        marks={x: date_list[x].strftime("%B %d, %Y") for x in range(numdays)},
        step=None
    )])

    
@app.callback(dash.dependencies.Output('map','figure'),[dash.dependencies.Input('year-slider','value'),dash.dependencies.Input('parameter','value')])
def update_figure(selected_date, param):
    print('Called')
    if(selected_date is not None):
        print('Clear 1')
        current_date = date_list[selected_date]
    if(param is not None):
        print('Clear 2')
        current_indicator[0] = param
        master_df = dataframes[current_indicator[-1]]    
    print(current_date)
    print(param)
    df = master_df[master_df['Date'] == current_date]
    print(df)
    trace= go.Choroplethmapbox(geojson=map_base,
                           locations=df['FIPS #'].astype(str),
                           z=df['value'],
                           colorscale='bluered',
                           colorbar_thickness=20,
                           text = df['County'],
                           marker_line_color='white',customdata = df['FIPS #'], hovertemplate = '<b>County</b>: <b>%{text}</b>'+
                                            '<br> <b>Val </b>: %{z}<br>'+  '<extra></extra>'
                        )
    fig= go.Figure(data = trace)
    mapboxt = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw'                           
                            
    fig.update_layout(title_text= 'Texas Heatmap',
                      title_x=0.5, width = 700, height=700,
                      mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
                                     accesstoken= mapboxt,style = 'basic',
                                     zoom=4.5,
                                   ));
    return fig
    
@app.callback(
    dash.dependencies.Output('text-content', 'children'),
    [dash.dependencies.Input('map', 'hoverData')])
def update_text(hoverData):
    if(hoverData == None):
        return "Hover over a county"
    s = df[df['FIPS #'] == hoverData['points'][0]['customdata']]
    # Blank county has blank /units/ of /parameter/
    print(current_indicator)
    return html.H3(
        '{} County has {} ppm of {}'.format(
            s.iloc[0]['County'],
            s.iloc[0]['value'],
            current_indicator[-1])
    )

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server(debug=True,use_reloader = False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 316-309-123
Debugger PIN: 316-309-123
Debugger PIN: 316-309-123
Debugger PIN: 316-309-123
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Called
Clear 1
Clear 2
2020-01-01 00:00:00
Ozone
            County       Date FIPS #     value
0             Bell 2020-01-01  48027  0.007047
191          Bexar 2020-01-01  48029  0.000364
382       Brazoria 2020-01-01  48039  0.007959
573       Brewster 2020-01-01  48043  0.004087
764        Cameron 2020-01-01  48061 -0.003403
955         Collin 2020-01-01  48085  0.004774
1146        Dallas 2020-01-01  48113  0.010249
1337        Denton 2020-01-01  48121  0.007910
1528       El Paso 2020-01-01  48141  0.007580
1719         Ellis 2020-01-01  48139  0.006139
1910     Galveston 2020-01-01  48167  0.014056
2101         G

['PM2.5']
['PM2.5']
['PM2.5']
['PM2.5']
['PM2.5']
['PM2.5']
['PM2.5']
['PM2.5']


In [ ]:
master_df